### Approximate SyncRank with sparsified Laplacian 


* 1. [MUN graph](#MUNgraph)
* 2. [ERO graph](#EROgraph)
* 3. [SBM MUN graph](#SBMMUNgraph)

In [85]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using Statistics
using Measures
using StatsBase#, JLD
using PlotlyJS

### 1. MUN graph  <a id="MUNgraph"></a>

Common parameters

In [86]:
methods = [
    "DPP(K) unif",
    "DPP(K) JL-LS",
    #"DPP(K) LS",
    "iid JL-LS",
    # "iid LS",
    "ST JL-LS",
    # "ST LS",
]

# simulation parameters
n_batch = 6 # maximum number of batches considered
n_rep = 3 # number of runs

# regularization parameter
q = 0
rng = Random.default_rng()

# graph parameters
n = 2000
p = 0.01


0.01

In [87]:
type = "MUN" 

eta = 1e-1# 1e-3

# planted ranking score
planted_score = randperm(rng,n)

# graph model
if type=="MUN"
    meta_g = gen_graph_mun(rng, n, p, eta;planted_score)
elseif type=="ERO"
    meta_g = gen_graph_ero(rng, n, p, eta;planted_score)
end

# storing settings
D_params = Dict("n" => n,"p" => p,"eta" => eta,"q" => q,"n_batch" => n_batch,"n_rep" => n_rep)
D_results = benchmark_syncrank(meta_g,planted_score, n_batch, n_rep, rng; methods)

# plotting distance between eigenvectors
check_connected = true
metric= "err"
y_limits = (1.5*1e-4, 1.4)
legendposition=:topright
plot_comparison_sync(metric,D_results,y_limits;legendposition,methods,check_connected)
name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

# saving plot
Plots.savefig("figures/"*name*".pdf")

# # saving results
# save("figures/"*name*"_data.jld", "D_results", D_results,"D_params", D_params)

In [ ]:
# plotting distance between eigenvectors
check_connected = true
metric= "err"
y_limits = (1.5*1e-9, 1.4)
legendposition=:topright
plot_comparison_sync(metric,D_results,y_limits;legendposition,methods,check_connected)
name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

In [ ]:
# plotting Kendall's tau between rankings

metric= "tau" 
y_limits = (0.4, 1.03)
legendposition=:bottomright
plot_comparison_sync(metric,D_results,y_limits;legendposition,methods)

name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)
# saving plot
Plots.savefig("figures/"*name*".pdf")

#### Number of cycles

In [ ]:
nb_samples = 100
nb_cycles = zeros(nb_samples)
weights_crsf = zeros(nb_samples)
for i_sample in 1:nb_samples
    mtsf = multi_type_spanning_forest(rng, meta_g, q)
    # check nb roots and cycles
    cycles = get_prop(mtsf, :cycle_nodes)
    nb_cycles[i_sample] = length(cycles)
    weights_crsf[i_sample] = get_prop(mtsf, :weight)
end    

cat_nb_cycles = string.(sort(Int.(nb_cycles)))
trace = PlotlyJS.histogram(x=cat_nb_cycles, histnorm="probability density")
layout = Layout(
    autosize=false,
    width=500,
    height=200,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=35,
    xaxis_type="category",
)
plt_cycles = PlotlyJS.plot(trace,layout)
name = "nb_cycles"*type*"eta"*string(eta)*"q"*string(q)
PlotlyJS.savefig(plt_cycles,"figures/"*name*".pdf")

In [ ]:
trace = box(y=weights_crsf,
boxpoints="all",
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)
name = "crsf_weight"*type*"eta"*string(eta)*"q"*string(q)
plt_weight = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt_weight,"figures/"*name*".pdf")

### 2. ERO graph  <a id="EROgraph"></a>

In [ ]:
type = "ERO" 

eta = 0.1
#eta = 0.02
# planted ranking score
planted_score = randperm(rng,n)

# graph model
if type=="MUN"
    meta_g = gen_graph_mun(rng, n, p, eta;planted_score)
elseif type=="ERO"
    meta_g = gen_graph_ero(rng, n, p, eta;planted_score)
end

# storing settings
D_params = Dict("n" => n,"p" => p,"eta" => eta,"q" => q,"n_batch" => n_batch,"n_rep" => n_rep)

check_connected = true
hkpv = false
D_results = benchmark_syncrank(meta_g,planted_score, n_batch, n_rep, rng; methods, hkpv)
# plotting distance between eigenvectors
metric= "err"
y_limits = (1.5*1e-4, 1.4)
legendposition=:bottomright
plot_comparison_sync(metric,D_results,y_limits;legendposition,methods,check_connected)
name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

# # saving plot
Plots.savefig("figures/"*name*".pdf")

# # saving results
# save("figures/"*name*"_data.jld", "D_results", D_results,"D_params", D_params)

In [ ]:
# plotting Kendall's tau between rankings
metric= "tau" 
y_limits = (0.1, 1.)
legendposition=:bottomright
plot_comparison_sync(metric,D_results,y_limits;legendposition,methods)

name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)
# saving plot
Plots.savefig("figures/"*name*".pdf")

In [ ]:
nb_samples = 100
nb_cycles = zeros(nb_samples)
weights_crsf = zeros(nb_samples)
for i_sample in 1:nb_samples
    mtsf = multi_type_spanning_forest(rng, meta_g, q)
    # check nb roots and cycles
    cycles = get_prop(mtsf, :cycle_nodes)
    nb_cycles[i_sample] = length(cycles)
    weights_crsf[i_sample] = get_prop(mtsf, :weight)
end    

cat_nb_cycles = string.(sort(Int.(nb_cycles)))
trace = PlotlyJS.histogram(x=cat_nb_cycles, histnorm="probability density")
layout = Layout(
    autosize=false,
    width=500,
    height=200,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=35,
    xaxis_type="category",
)
plt_cycles = PlotlyJS.plot(trace,layout)
name = "nb_cycles"*type*"eta"*string(eta)*"q"*string(q)
PlotlyJS.savefig(plt_cycles,"figures/"*name*".pdf")

In [ ]:
trace = box(y=weights_crsf,
boxpoints="all",
marker=attr(
        color="LightSkyBlue",
        size=1),
quartilemethod="linear",
name="",
marker_color="royalblue",
boxmean=true)

layout = Layout(
    autosize=false,
    width=500,
    height=500,
    margin=attr(l=1, r=1, b=1, t=1, pad=4),
    font_size=20,
)

name = "crsf_weight"*type*"eta"*string(eta)*"q"*string(q)
plt_weight = PlotlyJS.plot(trace,layout)
PlotlyJS.savefig(plt_weight,"figures/"*name*".pdf")

#### 3. SBM MUN graph  <a id="SBMMUNgraph"></a>

In [ ]:
# rng = Random.default_rng()

# # graph parameters
# n = 400
# p_in = 0.2
# p_out = 0.001
# η = 0.1
# #noise = 0.3

# # planted ranking score
# planted_score = randperm(rng,n)


# meta_g = MetaGraph(n)
# ic = false
# while !ic
#     if type=="MUN"
#         meta_g = ero_mun_sbm(rng,n,p_in,p_out,η,noise;planted_score)
#     elseif type=="ERO"
#         meta_g = ero_mun_sbm(rng,n,p_in,p_out,η,noise;planted_score)
#     end
#     ic = is_connected(meta_g)
# end
# println("connected graph generated with m/n = ", ne(meta_g) / nv(meta_g))



# # simulation parameters
# n_batch = 7 # maximum number of batches considered
# n_rep = 1 # number of runs

# # regularization parameter
# q = 0

# # # storing settings
# # D_params = Dict("n" => n,"p" => p,"eta" => eta,"q" => q,"n_batch" => n_batch,"n_rep" => n_rep)


# methods = [
#     "DPP(K) unif",
#     "DPP(K) JL-LS",
#     #"DPP(K) LS",
#     "iid JL-LS",
#     # "iid LS",
#     #"ST JL-LS",
#     # "ST LS",
# ]

# D_results = benchmark_syncrank(meta_g,planted_score, n_batch, n_rep, rng; methods)
# # plotting distance between eigenvectors
# metric= "err"
# y_limits = (1.5*1e-4, 1.4)
# legendposition=:topright
# plot_comparison_sync(metric,D_results,y_limits;legendposition,methods)
# name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

# # saving plot
# savefig("figures/"*name*".pdf")

In [ ]:
# # plotting Kendall's tau between rankings

# metric= "tau" 
# y_limits = (1.5*1e-4, 1.1)

# legendposition=:bottomright
# plot_comparison_sync(metric,D_results,y_limits;legendposition,methods)

# name = "syncrank_"*metric*"_"*type*"n"*string(n)*"p"*string(p)*"eta"*string(eta)*"q"*string(q)

# # saving plot
# savefig("figures/"*name*".pdf")